In [1]:
import requests                      # HTTP 요청을 보내기 위한 라이브러리
from PIL import Image                # 이미지 처리용 라이브러리 (Pillow)
import gradio as gr                  # Gradio UI 생성 라이브러리
import io                           # 바이트 스트림 처리를 위한 모듈

# Custom Vision Prediction 서비스 키와 엔드포인트 URL 설정
PREDICTION_KEY = "48fBztGiqcoH4PUUK23qmvxTBMmu31mgQ4JmJN1EDeCWNYw2QAtQJQQJ99BEACL93NaXJ3w3AAAIACOGelyc"
ENDPOINT_URL = "https://neptun91cv20250526-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/c387e65a-5db0-4e24-8b0a-3c049157dcc2/detect/iterations/Iteration1/image"

# API 호출에 사용할 헤더, Prediction-Key와 데이터 타입 지정
headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"  # 바이너리 이미지 데이터 전송
}

def predict_with_api(image: Image.Image):
    # PIL 이미지 객체를 JPEG 형식의 바이너리 데이터로 변환
    buf = io.BytesIO()               # 메모리 내 바이트 버퍼 생성
    image.save(buf, format='JPEG')  # 이미지 데이터를 JPEG로 버퍼에 저장
    byte_data = buf.getvalue()       # 버퍼에 저장된 바이너리 데이터 추출
    
    # Azure Custom Vision API에 POST 요청으로 이미지 전송
    response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)
    
    # JSON 형태로 응답 받아 예측 결과 파싱
    predictions = response.json()["predictions"]
    
    # 확률(probability)이 가장 높은 예측 항목을 선택
    top_prediction = max(predictions, key=lambda x: x["probability"])
    label = top_prediction["tagName"]           # 예측된 클래스 이름
    probability = top_prediction["probability"] # 예측 확률
    
    # 결과 문자열 포맷팅 후 반환 (예: 'cat (98.23%)')
    return f"{label} ({probability*100:.2f}%)"

# Gradio 인터페이스 구성
interface = gr.Interface(
    fn=predict_with_api,          # 이미지 입력을 받아 API 호출하는 함수 지정
    inputs=gr.Image(type="pil"), # 입력 타입: PIL 이미지 객체
    outputs=gr.Text(),            # 출력 타입: 텍스트 (예측 결과)
    title="Custom Vision Image Classifier",   # 앱 제목
    description="Upload an image to see the prediction from your Custom Vision model." # 앱 설명
)

# 웹 서버를 실행하여 인터페이스 실행 (브라우저에서 접속 가능)
interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
